# Datapipeline 과제

 data pipeline tutorial을 바탕으로 flower 데이터 세트 data pipeline 구현 및 훈련
 * flower dataset
     * daisy, dandelion, roses, sunflowers, tulips 5클래스
     * 다양한 크기의 3채널 jpg 파일
 * 데이터 경로 설정: data_dir flag를 사용하여 flower 데이터 세트가 저장된 경로 설정
 * 체크포인트 경로 설정: ckpt_name flag를 사용하여 tenosrflow 체크포인트가 저장될 경로 설정
 * 구현된 파이프 라인들간 성능 비교
 * 사용모델 tensorflow slim vgg 19 사용
     * model/slim_vgg.py 참조
 * train_gen.py: generator 기반 훈련 파일 참조
 
## 1. tf record 파일 생성
 flower dataset를 이용하여 tf record 파일 생성한다.
 * /data/make_tf_record.py
     * train 3320 파일, validation 350 파일로 나누어서 저장
     * ImageReader: jpeg 및 3 channel 지원하도록 코드수정
         * tf.image.decode_jpeg 함수 사용
     * _NUM_SHARDS = 5로 설정 후 아래 파일 생성
         * flowers_train_00000-of-00005.tfrecord
         * flowers_train_00001-of-00005.tfrecord
         * flowers_train_00002-of-00005.tfrecord
         * flowers_train_00003-of-00005.tfrecord
         * flowers_train_00004-of-00005.tfrecord
         * flowers_validation_00000-of-00005.tfrecord
         * flowers_validation_00001-of-00005.tfrecord
         * flowers_validation_00002-of-00005.tfrecord
         * flowers_validation_00003-of-00005.tfrecord
         * flowers_validation_00004-of-00005.tfrecord


## 2. tf.train.string_input_producer + tf.train.shuffle_batch 파이프라인 구성
 텐서플로우 string_input_producer 와 shuffle_batch 큐를 이용하여 prefetch data pipeline을 구성한다.<br/>
 /data/data_queue.py, train.py 파일을 작성 후,
 train.py 파일을 이용하여 flower 데이터 세트 훈련을 수행한다.
 * /data/data_queue.py
     * def inputs 함수 구현
         * input arg: split_name, batch_size, dataset_dir, num_epochs, width, height, is_training
         * output: images, labels
         * tf.train.string_input_producer 큐 생성
         * tf.train.string_input_producer 큐 read, decode
         * image preprocessing
             * inception_preprocessing.preprocess_image 함수 사용
         * tf.train.shuffle_batch 큐 생성
         * images, labels return
 * train.py
     * data_queue.py input 함수 사용하여 train pipeline 생성
         * split_name: "train"
         * is_training: True
     * data_queue.py input 함수 사용하여 validate pipeline 생성
         * split_name: "validation"
         * is_training: False
     * train pipeline 생성 후, 훈련 모델과 연결 후 훈련
     * train pipeline 1 epoch 수행 후 validate pipeline 1epoch 수행
         * validate_run 함수 내 validate pipeline 데이터 predict 수행
 * 2 epoch 수행 후, 훈련된 데이터 수 확인
 * step 당 수행 시간 확인
     * tf.train.shuffle_batch 큐 thread수 변경하며 수행시간 확인


## 3. tf.contrib.dataset 파이프라인 구성

 * /data/dataset_queue.py
      * def inputs 함수 구현
         * input arg: split_name, batch_size, dataset_dir, num_epochs, is_training
         * output: dataset
         * TFRecordDataset 파이프라인 생성
         * Dataset read_and_decode 함수 맵핑
         * is_training True: dataset_preprocessing.preprocess_for_train 함수 맵핑
         * is_training True: dataset_preprocessing.preprocess_for_eval 함수 맵핑
         * dataset 파이프라인 epoch 설정
         * dataset 파이프라인 batchsize 설정
         * dataset 파이프라인 return

 * /data/dataset_preprocessing.py
      * preprocess_for_train 함수 구현
          * inception_preprocessing.py preprocess_for_train 함수 기반 구현
          * input: image, label, height=224, width=224, bbox=None, fast_mode=True, scope=None
          * output: image, label 
          * height, width 지정
          * label input 및 output arg 추가
          
      * preprocess_for_eval 함수 구현
          * inception_preprocessing.py preprocess_for_eval 함수 기반 구현
          * input: image, label, height=224, width=224, central_fraction=0.875, scope=None
          * output: image, label 
          * height, width 지정
          * label input 및 output arg 추가

 * train_dataset.py
     * dataset_preprocessing.py input 함수 사용하여 train dataset 생성
         * split_name: "train"
         * is_training: True
     * dataset_preprocessing.py input 함수 사용하여 validate dataset 생성
         * split_name: "validation"
         * is_training: False
     * train dataset 생성 후, 훈련 모델과 연결 후 훈련
     * train dataset 1 epoch 수행 후 validate dataset 1epoch 수행
         * validate_run 함수 내 validate dataset 데이터 predict 수행
 * 2 epoch 수행 후, 훈련된 데이터 수 확인
 * step 당 수행 시간 확인


## 4. tf.contrib.dataset prefetch 파이프라인 구성 (추가과제)
 tf.contrib.dataset와 tf.train.shuffle_batch 큐를 이용하여 tf.contrib.dataset prefetch 파이프라인을 구성 
 - dataset 생성 후 dataset image,label tf.train.shuffle_batch 큐에 삽입
 * /data/dataset_queue_prefetch.py
     * def inputs 함수 구현
         * input arg: split_name, batch_size, dataset_dir, num_epochs, is_training
         * output: images, labels
         * TFRecordDataset 파이프라인 생성
         * Dataset read_and_decode 함수 맵핑
         * is_training True: dataset_preprocessing.preprocess_for_train 함수 맵핑
         * is_training True: dataset_preprocessing.preprocess_for_eval 함수 맵핑
         * dataset 파이프라인 epoch 설정
         * dataset 파이프라인 tf.train.shuffle_batch 큐 연결
         * images, labels 반환
 * train_dataset_prefetch.py
     * dataset_queue_prefetch.py input 함수 사용하여 train dataset 생성
         * split_name: "train"
         * is_training: True
     * dataset_queue_prefetch.py input 함수 사용하여 validate dataset 생성
         * split_name: "validation"
         * is_training: False
     * train dataset 생성 후, 훈련 모델과 연결 후 훈련
     * train dataset 1 epoch 수행 후 validate dataset 1epoch 수행
         * validate_run 함수 내 validate dataset 데이터 predict 수행
 * 2 epoch 수행 후, 훈련된 데이터 수 확인
 * step 당 수행 시간 확인
